# Chapter 8. Optimization for Training DeepModels

* 딥러닝 세미나 : 이론 [1]
* 김무성

# Contents
* 8.1 How Learning Diﬀers from Pure Optimization
    - 8.1.1 Empirical Risk Minimization
    - 8.1.2 Surrogate Loss Functions and Early Stopping
    - 8.1.3 Batch and Minibatch Algorithms
* 8.2 Challenges in Neural Network Optimization
    - 8.2.1 Ill-Conditioning
    - 8.2.2 Local Minima
    - 8.2.3 Plateaus, Saddle Points and Other Flat Regions
    - 8.2.4 Cliﬀs and Exploding Gradients
    - 8.2.5 Long-Term Dependencies
    - 8.2.6 Inexact Gradients
    - 8.2.7 Poor Correspondence between Local and Global Structure
    - 8.2.8 Theoretical Limits of Optimization
* 8.3 Basic Algorithms
    - 8.3.1 Stochastic Gradient Descent
    - 8.3.2 Momentum
    - 8.3.3 Nesterov Momentum
* 8.4 Parameter Initialization Strategies
* 8.5 Algorithms with Adaptive Learning Rates
    - 8.5.1 AdaGrad
    - 8.5.2 RMSProp
    - 8.5.3 Adam
    - 8.5.4 Choosing the Right Optimization Algorithm
* 8.6 Approximate Second-Order Methods
    - 8.6.1 Newton’s Method
    - 8.6.2 Conjugate Gradients
    - 8.6.3 BFGS
* 8.7 Optimization Strategies and Meta-Algorithms
    - 8.7.1 Batch Normalization
    - 8.7.2 Coordinate Descent
    - 8.7.3 Polyak Averaging
    - 8.7.4 Supervised Pretraining
    - 8.7.5 Designing Models to Aid Optimization
    - 8.7.6 Continuation Methods and Curriculum Learning

#### 참고
* [2] An overview of gradient descent optimization algorithms - http://sebastianruder.com/optimizing-gradient-descent/index.html
* [3] Stochastic gradient methods for machine learning - http://research.microsoft.com/en-us/um/cambridge/events/mls2013/downloads/stochastic_gradient.pdf
* [4] (OXFORD, Machine Learning: 2014-2015 )Deep Learning Lecture 6: Optimization - https://www.cs.ox.ac.uk/people/nando.defreitas/machinelearning/lecture5.pdf
* [5] 다크프로그래머 : Gradient, Jacobian 행렬, Hessian 행렬, Laplacian- http://darkpgmr.tistory.com/132
* [6] 조금은 느리게 살자 : 테일러 급수(級數, Taylor series) - http://ghebook.blogspot.kr/2010/07/taylor-series.html
* [7] 다크프로그래머 : 테일러 급수의 이해와 활용 (Taylor series) - http://darkpgmr.tistory.com/59

This chapter presents these optimization techniques for neural network training.

<img src="https://qph.is.quoracdn.net/main-qimg-46a77c77c721ba34283308232a1788c8?convert_to_webp=true" width=600 />

<img src="https://msampler.files.wordpress.com/2009/07/cvx-fun.gif" width=600 />

<img src="https://camo.githubusercontent.com/30bf2d42d3a9b0e07dbc03a014f4e36dbc06904f/68747470733a2f2f7261772e6769746875622e636f6d2f7175696e6e6c69752f4d616368696e654c6561726e696e672f6d61737465722f696d61676573466f724578706c616e6174696f6e2f4772616469656e7444657363656e74576974684d75746c69706c654c6f63616c4d696e696d756d2e6a7067" width=600 />

<img src="http://image.slidesharecdn.com/aihandson20140710bslideshare-140716182318-phpapp02/95/jsais-ai-tool-introduction-deep-learning-pylearn2-and-torch7-52-638.jpg?cb=1435207500" width=600 />

<font color="red">This chapter focuses on one particular case of optimization</font>: 
* ﬁnding the parameters $θ$ of a neural network that signiﬁcantly reduce a cost function $J(θ)$, which typically includes a performance measure evaluated on the entire training set as well as additional regularization terms.
    <img src="http://cs231n.github.io/assets/dataflow.jpeg" />
* We begin with a description of how optimization used as a training algorithm for a machine learning task diﬀers from pure optimization. 
* Next, we present several of the concrete challenges that make optimization of neural networks diﬃcult. 
* We then deﬁne several practical algorithms, including both
    - optimization algorithms themselves and 
    - strategies for initializing the parameters. 
* More advanced algorithms adapt 
    - their learning rates during training or 
    - leverage information contained in the second derivatives of the cost function
* Finally, we conclude with a review of 
    - several optimization strategies that are formed by 
        - combining simple optimization algorithms 
            - into higher-level procedures.

# 8.1 How Learning Diﬀers from Pure Optimization
* 8.1.1 Empirical Risk Minimization
* 8.1.2 Surrogate Loss Functions and Early Stopping
* 8.1.3 Batch and Minibatch Algorithms

Optimization algorithms used for training of deep models diﬀer from traditional optimization algorithms in several ways.
* Machine learning usually acts indirectly.
    - In most machine learning scenarios, 
        - we care about some performance measure $P$, 
            - that is deﬁned with respect to the test set and 
            - may also be intractable. 
    - We therefore optimize $P$ only indirectly. 
* We reduce a diﬀerent cost function $J(θ)$ in the hope that 
    - doing so will improve P. 
* This is in contrast to pure optimization,where minimizing 
    - $J$ is a goal in and of itself. 
* Optimization algorithms for training deep models 
    - also typically include 
        - some specialization 
            - on the speciﬁc structure 
                - of machine learning objective functions.

Typically, the cost function can be written as an average over the training set,such as

<img src="figures/cap8.1.png" width=600 />

whereLis the per-example loss function, $f(x;θ)$ is the predicted output whenthe input is $x$, $ˆp$ data is the empirical distribution.

#### data generating distribution

* Eq. 8.1 deﬁnes an objective function with respect to the training s
* We would usually prefer to minimize 
    - the corresponding objective function 
        - where the expectation is taken across 
            - the data generating distribution $p$ data 
                - rather than just over the ﬁnite training set:

<img src="figures/cap8.2.png" width=600 />

## 8.1.1 Empirical Risk Minimization

#### 참고
* [8] Empirical Risk Minimization - http://demo.clab.cs.cmu.edu/fa2015-11763/slides/erm.pdf
* [9] The Learning Problem and Regularization - http://www.mit.edu/~9.520/spring11/slides/class02.pdf
* [10] Risk Minimization - http://hellbell.tistory.com/entry/Risk-Minimization

<img src="http://www.svms.org/srm/Rychetsky2001_2-4.png" />

The simplest way to convert a machine learning problem back into an optimization problem is to minimize the expected loss on the training set. 
* This means replacing the true distribution $p(x, y)$ with the empirical distribution $ˆp(x, y)$ deﬁned by the training set. 
* We now minimize the empirical risk

<img src="figures/cap8.3.png" width=600 />

where m is the number of training examples.

The training process based on minimizing this average training error is known as <font color="red">empirical risk minimization</font>.
* In this setting, machine learning is still very similarto straightforward optimization. 
* Rather than optimizing the risk directly, 
    - we optimize the empirical risk, 
    - and hope that 
        - the risk decreases signiﬁcantly as well.
* A variety of theoretical results establish conditions under which the true risk can be expected to decrease by various amounts

<font color="red">However, empirical risk minimization is prone to overﬁtting</font>. 
* Models with high capacity can simply memorize the training set. In many cases, empirical risk minimization is not really feasible. 
    - The most eﬀective modern optimizationalgorithms are based on gradient descent, but many useful loss functions, suchas 0-1 loss, have no useful derivatives (the derivative is either zero or undeﬁnedeverywhere). 
* These two problems mean that, <font color="red">in the context of deep learning, werarely use empirical risk minimization</font>. 
* Instead, we must use a slightly diﬀerent approach, in which the quantity that we actually optimize is even more diﬀerent from the quantity that we truly want to optimize

## 8.1.2 Surrogate Loss Functions and Early Stopping

#### 참고
* [11] Surrogate Loss Functions in Machine Learning - http://fa.bianp.net/blog/2014/surrogate-loss-functions-in-machine-learning/

#### Surrogate Loss Functions

Sometimes, the loss function we actually care about (say classiﬁcation error) is notone that can be optimized eﬃciently. 
* For example, exactly minimizing expected 0-1 loss is typically intractable (exponential in the input dimension), even for a linearclassiﬁer (Marcotte and Savard, 1992). 

In such situations, one typically optimizesa <font color="red">surrogate loss function</font> instead, which acts as a proxy but has advantages.
* For example, the negative log-likelihood of the correct class is typically used as asurrogate for the 0-1 loss. 
* The negative log-likelihood allows the model to estimatethe conditional probability of the classes, given the input, and if the model can do that well, then it can pick the classes that yield the least classiﬁcation error in expectation.

<img src="http://fa.bianp.net/blog/images/2014/loss_functions.png" width=600 />

<img src="http://fa.bianp.net/blog/images/2014/loss_01.png" width=600 />

<img src="http://fa.bianp.net/blog/images/2014/loss_log.png" width=600 />

#### Early Stopping

* A very important diﬀerence between optimization in general and optimizationas we use it for training algorithms is that training algorithms do not usually haltat a local minimum. 
* Instead, a machine learning algorithm usually minimizesa surrogate loss function but halts when a convergence criterion based on early stopping (Sec. 7.8) is satisﬁed.  
* <font color="red">Typically the early stopping criterion is based on the true underlying loss function, such as 0-1 loss measured on a validation set</font>, and is designed to cause the algorithm to halt whenever overﬁtting begins to occur. 

<img src="http://documentation.statsoft.com/STATISTICAHelp/SANN/Images/ErrVsTrain.bmp"  />

<img src="http://deeplearning4j.org/img/earlystopping.png" width=600 />

## 8.1.3 Batch and Minibatch Algorithms

#### 참고
* [12] Computer vision: models, learning and inference / Chapter 4 Fitting Probability Models. - http://slideplayer.com/slide/5277039/

Optimization algorithms for machine learning typically compute each update to the parameters based on an <font color="red">expected value of the cost function</font> <font color="blue">estimated using only a subset of</font> the terms of the full cost function.

<img src="http://www.onekobo.com/Articles/Statistics/statsImgs/image204c.gif" width=600 />

For example, maximum likelihood estimation problems, when viewed in log space, decompose into a sum over each example:

<img src="figures/cap8.4.png" width=600 />

<img src="http://images.slideplayer.com/17/5277039/slides/slide_47.jpg" width=600 />

Maximizing this sum is equivalent to maximizing the expectation over the <font color="red">empirical distribution</font> deﬁned by the training set:

<img src="figures/cap8.5.png" width=600 />

<img src="https://www.unc.edu/courses/2007spring/enst/562/001/images/lectures/lecture25/fig2.png" />

Most of the properties of the objective function $J$ used by most of our optimization algorithms are also <font color="red">expectations over the training set</font>.

For example, the most commonly used property is the gradient:

<img src="figures/cap8.6.png" width=600 />

<font color="red">Computing this expectation exactly is very expensive because it requires evaluating the model on every example in the entire dataset</font>.

#### more samples ? small number of samples ?
* Recall that the standard error of the mean (Eq. 5.46) estimated from n samples is given by σ/√n, where σ is the true standard deviation of the value of the samples.
* The denominator of √n shows that there are less than linear returns to using more examples to estimate the gradient. 
* Compare two hypothetical estimates of the gradient, one based on 100 examples and another based on 10,000 examples.
    - <font color="red">The latter requires 100 times more computation than the former, but reduces the standard error of the mean only by a factor of 10</font>.

<font color="blue">Most optimization algorithms converge much faster (in terms of total computation, not in terms of number ofupdates) if they are allowed to rapidly compute approximate estimates of the gradient rather than slowly computing the exact gradient.</font>

Another consideration motivating statistical estimation of the gradient from a small number of samples is <font color="red">redundancy in the training set</font>.
* In the worst case, all $m$ samples in the training set could be identical copies of each other. 
* A sampling-based estimate of the gradient could compute the correct gradient with a single sample, using $m$ times less computation than the naive approach.

#### batch (or deterministic)

Optimization algorithms that use the entire training set are called batch ordeterministic gradient methods, <font color="red">because they process all of the training examples simultaneously in a large batch</font>.

<img src="http://image.slidesharecdn.com/aihandson20140710bslideshare-140716182318-phpapp02/95/jsais-ai-tool-introduction-deep-learning-pylearn2-and-torch7-52-638.jpg?cb=1435207500" width=600 />

#### minibatch

Typically the term “batch gradient descent”implies the use of the full training set, while the use of the term “batch” to describea group of examples does not. For example, it is very common to use the term“batch size” to describe the size of a minibatch.

#### stochastic (or online)

* Optimization algorithms that use only a single example at a time are sometimescalled stochastic or sometimes online methods. 
* The term online is usually reservedfor the case where the examples are drawn from a stream of continually createdexamples rather than from a ﬁxed-size training set over which several passes aremade.

#### minibatch or minibatch stochastic methods or stochastic methods.

Most algorithms used for deep learning fall somewhere in between, using more one but less than all of the training examples. 
* These were traditionally called minibatch or minibatch stochastic methods and it is now common to simply call them stochastic methods.

Minibatch sizes are generally driven by the following factors:
* Larger batches provide a more accurate estimate of the gradient, but with less than linear returns.
* Multicore architectures are usually underutilized by extremely small batches.
    - This motivates using some absolute minimum batch size, below which thereis no reduction in the time to process a minibatch.
* If all examples in the batch are to be processed in parallel (as is typically the case), then the amount of memory scales with the batch size. 
    - For many hardware setups this is the limiting factor in batch size.
* Some kinds of hardware achieve better runtime with speciﬁc sizes of arrays.
    - Especially when using GPUs, it is common for power of 2 batch sizes to oﬀer better runtime. 
    - Typical power of 2 batch sizes range from 32 to 256, with 16 sometimes being attempted for large models.
* Small batches can oﬀer a regularizing eﬀect (Wilson and Martinez, 2003),perhaps due to the noise they add to the learning process. 
    - Generalization error is often best for a batch size of 1.
    - Training with such a <font color="red">small batch size</font> might require a <font color="red">small learning rate</font> to maintain stability due to the high variance in the estimate of the gradient. 
    - <font color="red">The total runtime can be very high</font> due to the need to make more steps, both because of the reduced learning rate and because it takes more steps to observe the entire training set.

Diﬀerent kinds of algorithms use <font color="blue">diﬀerent kinds of information from the mini-batch</font> in diﬀerent ways.
* Some algorithms are more sensitive to sampling error than others, either 
    - because they use information that is diﬃcult to estimate accurately with few samples, or 
    - because they use information in ways that amplify sampling errors more.

#### gradient only
* Methods that compute updates based only on the gradient $g$ are usually relatively robust and can handle smaller batch sizes like 100.

#### Second-ordermethods
* Second-ordermethods, which use also the Hessian matrixHand compute updates such as $H^{−1}g$, typically require much larger batch sizes like 10,000.

#### minibatches be selected randomly
* It is also crucial that the minibatches be selected randomly.
    - Computing an unbiased estimate of the expected gradient from a set of samples requires that those samples be independent.
    - We also wish for two subsequent gradient estimates to be independent from each other, so two subsequent minibatches of examples should also be independent from each other. 
    - Many datasets are most naturally arranged in a way where successive examples are highly correlated.
* For very large datasets, for example datasets containing billions of examples in a data center, it can be impractical to sample examples truly uniformlyat random every time we want to construct a minibatch.
    - <font color="red">Fortunately, in practiceit is usually suﬃcient to shuﬄe the order of the dataset once and then store it in shuﬄed fashion.</font>

#### parallelism

* Many optimization problems in machine learning decompose over examples well enough that we can compute entire separate updates over diﬀerent examples in parallel. 
* In other words, we can compute the update that minimizes $J(X)$ for one minibatch of examples $X$ at the same time that we compute the update for several other minibatches. 
* Such asynchronous parallel distributed approaches are discussed further in Sec. 12.1.3.

<img src="http://docplayer.net/docs-images/26/7315497/images/71-0.png" width=600 />

#### generalization error

* An interesting motivation for minibatch stochastic gradient descent is that it follows the gradient of the true generalization error(Eq. 8.2) so long as <font color="red">no examples are repeated</font>.
    - Most implementations of minibatch stochastic gradient descent shuﬄe the dataset once and then pass through it multiple times. 
        - On the ﬁrst pass, each minibatch is used to compute an unbiased estimate of the true generalization error. 
        - On the second pass, the estimate becomes biased because it isformed by re-sampling values that have already been used, rather than obtaining new fair samples from the data generating distribution.
* The fact that stochastic gradient descent minimizes generalization error iseasiest to see in the online learning case, where examples or minibatches are drawnfrom a stream of data. 
    - In other words, instead of receiving a ﬁxed-size training set, the learner is similar to a living being who sees a new example at each instant, with every example (x, y) coming from the data generating distribution $p_{data}(x, y)$.
    - In this scenario, examples are never repeated; every experience is a fair sample from $p_{data}$.

<img src="figures/cap8.7.png" width=600 />

<img src="figures/cap8.8.png" width=600 />

# 8.2 Challenges in Neural Network Optimization
* 8.2.1 Ill-Conditioning
* 8.2.2 Local Minima
* 8.2.3 Plateaus, Saddle Points and Other Flat Regions
* 8.2.4 Cliﬀs and Exploding Gradients
* 8.2.5 Long-Term Dependencies
* 8.2.6 Inexact Gradients
* 8.2.7 Poor Correspondence between Local and Global Structure
* 8.2.8 Theoretical Limits of Optimization

#### 참고
* [13] ICML 2016 tutorials - http://icml.cc/2016/?page_id=97
* [14] Recent Advances in Non-Convex Optimization - http://newport.eecs.uci.edu/anandkumar/slides/icml2016-tutorial.pdf

When training neural networks, we must confront the general non-convex case.

 In this section,we summarize several of the most prominent challenges involved in optimization for training deep models.

## 8.2.1 Ill-Conditioning

#### 참고
* [15] 조건수(condition number) -  https://ko.wikipedia.org/wiki/%EC%A1%B0%EA%B1%B4%EC%88%98
* [5] 다크프로그래머 : Gradient, Jacobian 행렬, Hessian 행렬, Laplacian- http://darkpgmr.tistory.com/132
* [6] 조금은 느리게 살자 : 테일러 급수(級數, Taylor series) - http://ghebook.blogspot.kr/2010/07/taylor-series.html
* [7] 다크프로그래머 : 테일러 급수의 이해와 활용 (Taylor series) - http://darkpgmr.tistory.com/59

Some challenges arise even when optimizing convex functions. Of these, the most prominent is <font color="red">ill-conditioning of the Hessian matrix $H$ </font>.

The ill-conditioning problem is generally believed to be present in <font color="red">neural network training problems</font>. 

Ill-conditioning can manifest by <font color="red">causing SGD to get “stuck”</font> in the sense that even very small steps increase the cost function.

<img src="figures/cap8.9.png" width=600 />

<img src="figures/cap8.10.png" width=600 />

<img src="figures/cap8.11.png" width=600 />

#### Newton's method

* Though ill-conditioning is present in other settings besides neural network training, some of the techniques used to combat it in other contexts are less applicable to neural networks. 
* For example, Newton’s method is an excellent tool for minimizing convex functions with poorly conditioned Hessian matrices, 
    - but in the subsequent sections we will argue that Newton’s method <font color="red">requires signiﬁcant modiﬁcation before it can be applied to neural networks</font>.

## 8.2.2 Local Minima

<img src="https://www.safaribooksonline.com/library/view/fundamentals-of-deep/9781491925607/assets/deep_local_minima.png" width=600 />

#### 참고
[16] The Challenges with Gradient Descent - https://www.safaribooksonline.com/library/view/fundamentals-of-deep/9781491925607/ch04.html

* When optimizing a convex function, we know that we have reached a good solution if we ﬁnd a critical point of any kind.
* <font color="red">With non-convex functions</font>, such as neural nets, it is possible to have <font color="red">many local minima</font>. Indeed, nearly any deep model is essentially guaranteed to havean extremely large number of local minima. However, as we will see, this is not necessarily a major problem.

#### model identiﬁability problem
Neural networks and any models with multiple equivalently parametrized latent variables all have multiple local minima because of the model identiﬁability problem.
* A model is said to be identiﬁable if a suﬃciently large training set can rule out all but one setting of the model’s parameters.

<img src="https://www.safaribooksonline.com/library/view/fundamentals-of-deep/9781491925607/assets/rearrangement_invariance.png" width=600 />

#### weight space symmetry
Models with latent variables are often not identiﬁable because we can obtain equivalent models by exchanging latent variables with each other. 
* For example, we could take a neural network and modify layer 1 by swapping the incoming weight vector for unit $i$ with the incoming weight vector for unit $j$, then doing the same for the outgoing weight vectors. 
* If we have $m$ layers with $n$ units each, then there are $n!^{m}$ ways of arranging the hidden units.
* This kind of <font color="red">non-identiﬁability is known as weight space symmetry</font>.

#### non-identiﬁability
In addition to weight space symmetry, <font color="red">many kinds of neural networks have additional causes of non-identiﬁability</font>. 
* For example, in any rectiﬁed linear or maxout network, we can scale all of the incoming weights and biases of a unit by $α$ if we also scale all of its outgoing weights by $1/α$. 
* This means that — <font color="red">if the cost function does not include terms such as weight decay that depend directly on the weights rather than the models’ outputs</font>
    - every local minimum of a rectiﬁed linear or maxout network lies on an (m × n)-dimensional hyperbola of equivalent local minima

#### local minima are not a problematic form of non-convexity

* <font color="blue">These model identiﬁability issues mean that there can be an extremely large or even uncountably inﬁnite amount of local minima in a neural network cost function.</font>
* However, all of these local minima arising from non-identiﬁability are equivalent to each other in cost function value. As a result, <font color="red">these local minima are not a problematic form of non-convexity</font>.

#### Local minima can be problematic if 
*  Local minima can be problematic if 
    - they have high cost in comparison to the global minimum. 
* One can construct small neural networks, 
    - even without hidden units, 
    - that have local minima with higher cost 
        - than the global minimum
* <font color="red">If local minima with high cost are common, this could pose a serious problem for gradient-based optimization algorithms</font>

#### open question

* It remains an open question 
    - whether there are many local minima of high cost for networks of practical interest and 
    - whether optimization algorithms encounter them.

#### local minima problem
* For many years, most practitioners believed that local minima were a common problem plaguing neural network optimization. 
* <font color="red">Today, that does not appear to be the case</font>. 
* The problem remains an active area of research, but experts now suspect that, 
    - for suﬃciently large neural networks, 
    - most local minima have a low cost function value, 
    - and that it is not important to ﬁnd a true global minimum
    - <font color="red">rather than to ﬁnd a point in parameter space that has low but not minimal cost</font>.

<img src="http://nbviewer.jupyter.org/github/KonanAcademy/deep/blob/master/seminar/ch06/figures/fig4.3.png" width=600 />

## 8.2.3 Plateaus, Saddle Points and Other Flat Regions

<img src="https://www.safaribooksonline.com/library/view/fundamentals-of-deep/9781491925607/assets/1d_critical_points.png" width=600 />

#### saddle point

For many high-dimensional non-convex functions, local minima (and maxima)are in fact rare compared to another kind of point with zero gradient: a saddle point. 
* Some points around a saddle point have greater cost than the saddle point, while others have a lower cost.

#### Hessian matrix
At a saddle point, 
* the Hessian matrix has 
    - both positive and negative eigenvalues. 
* Points lying along eigenvectors associated with positive eigenvalues have greater cost than the saddle point, 
    - while points lying along negative eigenvalues have lower value.

<img src="https://www.safaribooksonline.com/library/view/fundamentals-of-deep/9781491925607/assets/saddle_point.png" width=600 />

<img src="figures/cap8.12.png" width=600 />

## 8.2.4 Cliﬀs and Exploding Gradients

<img src="figures/cap8.13.png" width=600 />

## 8.2.5 Long-Term Dependencies

<img src="figures/cap8.14.png" width=600 />

## 8.2.6 Inexact Gradients

## 8.2.7 Poor Correspondence between Local and Global Structure

<img src="figures/cap8.15.png" width=600 />

<img src="figures/cap8.16.png" width=600 />

## 8.2.8 Theoretical Limits of Optimization

# 8.3 Basic Algorithms
* 8.3.1 Stochastic Gradient Descent
* 8.3.2 Momentum
* 8.3.3 Nesterov Momentum

## 8.3.1 Stochastic Gradient Descent

<img src="figures/cap8.17.png" width=600 />

<img src="figures/cap8.18.png" width=600 />

<img src="figures/cap8.19.png" width=600 />

## 8.3.2 Momentum

<img src="figures/cap8.20.png" width=600 />

<img src="figures/cap8.21.png" width=600 />

<img src="figures/cap8.22.png" width=600 />

<img src="figures/cap8.23.png" width=600 />

<img src="figures/cap8.24.png" width=600 />

<img src="figures/cap8.25.png" width=600 />

<img src="figures/cap8.26.png" width=600 />

## 8.3.3 Nesterov Momentum

<img src="figures/cap8.27.png" width=600 />

<img src="figures/cap8.28.png" width=600 />

# 8.4 Parameter Initialization Strategies

<img src="figures/cap8.29.png" width=600 />

<img src="figures/cap8.30.png" width=600 />

# 8.5 Algorithms with Adaptive Learning Rates
* 8.5.1 AdaGrad
* 8.5.2 RMSProp
* 8.5.3 Adam
* 8.5.4 Choosing the Right Optimization Algorithm

## 8.5.1 AdaGrad

<img src="figures/cap8.31.png" width=600 />

## 8.5.2 RMSProp

<img src="figures/cap8.32.png" width=600 />

<img src="figures/cap8.33.png" width=600 />

## 8.5.3 Adam

<img src="figures/cap8.35.png" width=600 />

## 8.5.4 Choosing the Right Optimization Algorithm

<img src="figures/cap8.34.png" width=600 />

# 8.6 Approximate Second-Order Methods
* 8.6.1 Newton’s Method
* 8.6.2 Conjugate Gradients
* 8.6.3 BFGS

## 8.6.1 Newton’s Method


<img src="figures/cap8.36.png" width=600 />

<img src="figures/cap8.37.png" width=600 />

<img src="figures/cap8.38.png" width=600 />

## 8.6.2 Conjugate Gradients


<img src="figures/cap8.40.png" width=600 />

<img src="figures/cap8.39.png" width=600 />

<img src="figures/cap8.41.png" width=600 />

<img src="figures/cap8.42.png" width=600 />

<img src="figures/cap8.43.png" width=600 />

## 8.6.3 BFGS

<img src="figures/cap8.44.png" width=600 />

<img src="figures/cap8.45.png" width=600 />

<img src="figures/cap8.46.png" width=600 />

### Limited Memory BFGS (or L-BFGS)

# 8.7 Optimization Strategies and Meta-Algorithms
* 8.7.1 Batch Normalization
* 8.7.2 Coordinate Descent
* 8.7.3 Polyak Averaging
* 8.7.4 Supervised Pretraining
* 8.7.5 Designing Models to Aid Optimization
* 8.7.6 Continuation Methods and Curriculum Learning

## 8.7.1 Batch Normalization

<img src="figures/cap8.47.png" width=600 />


<img src="figures/cap8.48.png"  />

<img src="figures/cap8.49.png" />

<img src="figures/cap8.50.png" width=600 />

<img src="figures/cap8.51.png" width=600 />

<img src="figures/cap8.52.png" width=600 />

## 8.7.2 Coordinate Descent

<img src="figures/cap8.53.png" width=600 />

## 8.7.3 Polyak Averaging

<img src="figures/cap8.54.png" width=600 />

## 8.7.4 Supervised Pretraining

<img src="figures/cap8.55.png" width=600 />

<img src="figures/cap8.56.png" width=600 />

## 8.7.5 Designing Models to Aid Optimization


## 8.7.6 Continuation Methods and Curriculum Learning

<img src="figures/cap8.57.png" width=600 />

# 참고자료

* [1] DEEP LEARNING (Yoshua Bengio) : 8. Optimization for Training Deep Models - http://www.deeplearningbook.org/contents/optimization.html
* [2] An overview of gradient descent optimization algorithms - http://sebastianruder.com/optimizing-gradient-descent/index.html
* [3] Stochastic gradient methods
for machine learning -  http://research.microsoft.com/en-us/um/cambridge/events/mls2013/downloads/stochastic_gradient.pdf
* [4] (OXFORD, Machine Learning: 2014-2015
)Deep Learning Lecture 6: Optimization - https://www.cs.ox.ac.uk/people/nando.defreitas/machinelearning/lecture5.pdf
* [5] 다크프로그래머 : Gradient, Jacobian 행렬, Hessian 행렬, Laplacian- http://darkpgmr.tistory.com/132
* [6] 조금은 느리게 살자 : 테일러 급수(級數, Taylor series) - http://ghebook.blogspot.kr/2010/07/taylor-series.html
* [7] 다크프로그래머 : 테일러 급수의 이해와 활용 (Taylor series) - http://darkpgmr.tistory.com/59
* [8] Empirical Risk Minimization - http://demo.clab.cs.cmu.edu/fa2015-11763/slides/erm.pdf
* [9] The Learning Problem and Regularization - http://www.mit.edu/~9.520/spring11/slides/class02.pdf
* [10] Risk Minimization - http://hellbell.tistory.com/entry/Risk-Minimization
* [11] Surrogate Loss Functions in Machine Learning - http://fa.bianp.net/blog/2014/surrogate-loss-functions-in-machine-learning/
* [12] Computer vision: models, learning and inference / Chapter 4 Fitting Probability Models. - http://slideplayer.com/slide/5277039/
* [13] ICML 2016 tutorials - http://icml.cc/2016/?page_id=97
* [14] Recent Advances in Non-Convex Optimization - http://newport.eecs.uci.edu/anandkumar/slides/icml2016-tutorial.pdf
* [15] 조건수(condition number) -  https://ko.wikipedia.org/wiki/%EC%A1%B0%EA%B1%B4%EC%88%98
* [16] The Challenges with Gradient Descent - https://www.safaribooksonline.com/library/view/fundamentals-of-deep/9781491925607/ch04.html